In [1]:
import pandas as pd

# DataSet de départ
data = pd.read_csv("data.csv", encoding="iso-8859-1")

# DataSet des Top Hit (jusqu'à 2017)
top_songs = pd.read_csv("top_data.csv", encoding="iso-8859-1")

# On filtre donc notre DataSet de départ pour conserver seuelement les musiques avec date > 2017
data_2017 = data[data["year"]<2017]


# On recherche l'existence des musiques dans le DataSet des Hit Songs pour créer une nouvelle variable de succès 
data_2017["top"] = data_2017["name"].isin(top_songs["title"])
data_2017["top"] = data_2017["top"].astype(int)



C:\Users\diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [2]:
# Création de datasets en fonction des années
def split_df_by_year(df, range_max, year):
    """
        df : dataframe d'entrée
        step : range maximum d'annees que l'on veut pout notre dataset
        year : annnee de sortie de la musique à preédire
        Renvoie le dataframe respectant le range_max et l'annee
    """

    mask = (df['year'] <= year) & (df['year'] > year - range_max)
    current_df = df.loc[mask]
        
        
    return current_df
         
df = split_df_by_year(data_2017, 5, 2015)
print(df["year"].value_counts())


2014    2000
2013    2000
2012    2000
2011    2000
2015    1931
Name: year, dtype: int64


In [ ]:
print(data_2017.columns)
data_2017

In [ ]:
! pip install keras_pandas

In [10]:
### Predict Top Hit ###

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.over_sampling import SMOTE
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import StandardScaler

#input_data = data_2017
input_data = df
input_data["duration_ms"] = input_data["duration_ms"].astype(float)
input_data["explicit"] = input_data["explicit"].astype(float)
input_data["key"] = input_data["key"].astype(float)
input_data["mode"] = input_data["mode"].astype(float)

# undersampling
#df_ok = input_data[input_data['top'] == 1]
#df_ko = input_data.sample(df_ok.shape[0])
#input_data = pd.concat([df_ok, df_ko])

# Sample to avoid oversampling
df_ok = input_data[input_data['top'] == 1]
df_ko = input_data.sample(df_ok.shape[0])
input_data = pd.concat([df_ok, df_ko])

#print(input_data.shape)

# Preprocessing des datas
train_data, test_data = train_test_split(input_data, test_size=0.2)

#x_train = train_data.drop(["popularity", "id", "release_date", "name", "artists", "instrumentalness",
                          # "key", "tempo", "loudness", "year", "duration_ms", "explicit", "mode"], axis=1)
#y_train = x_train.pop('top')

#x_test = test_data.drop(["popularity", "id", "release_date", "name", "artists", "key", "instrumentalness",
                        # "tempo", "loudness", "year", "duration_ms", "explicit", "mode"], axis=1)
#y_test = x_test.pop('top')


x_train = train_data['acousticness']
y_train = train_data['top']

x_test = train_data['acousticness']
y_test = train_data['top']


# Undersampling
#rus = RandomUnderSampler()
#x_train, y_train = rus.fit_sample(x_train, y_train)

# Oversampling
#sm = SMOTE()
#x_train, y_train = sm.fit_resample(x_train, y_train)
#print(x_train.shape)


# List out variable types  (pas utilse ici mais on sait jamais)
data_type_dict = {'numerical': ['instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', "instrumentalness",
                                'tempo', 'valence', 'acousticness', 'danceability', 'duration_ms', 'energy'],
                  'categorical': ['mode', 'year', 'explicit'],
                  'text': ['name', 'artists']}
output_var = 'top'


#sc = StandardScaler()
#sc.fit(x_train)
#x_train = sc.transform(x_train)
#x_test = sc.transform(x_test)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))




# Define the model
model = keras.Sequential()
tf.keras.backend.set_floatx('float64')
tf.keras.Input(shape=1,)
#model.add(Dense(6, activation='sigmoid'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2))
#model.add(Dense(1, activation='sigmoid'))


plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

#lrate =0.01
#model.compile(loss='categorical_crossentropy',
#          optimizer=keras.optimizers.SGD(learning_rate= lrate),
#          metrics =['accuracy'])

model.compile(loss='binary_crossentropy', 
              optimizer=keras.optimizers.SGD(learning_rate= 0.1), 
              metrics=['accuracy']
             )


#[print(i.shape, i.dtype) for i in model.inputs]
#[print(o.shape, o.dtype) for o in model.outputs]
#[print(l.name, l.input_shape, l.dtype) for l in model.layers]
#model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))
#model.fit(x_train, y_train, validation_data=(x_test, y_test))
model.fit(x_train, y_train)

prediction = model.predict_classes(x_test)
#print(prediction)
#prediction = model.predict_proba(x_test)

loss, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

C:\Users\diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\diego\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
126/126 [==============================] - 0s 1ms/step - loss: 7.7125 - accuracy: 0.6320
Accuracy: 63.20


In [11]:

#print(len(prediction))
#print(y_test)
#print(prediction)
#print(y_test)
cm1 = confusion_matrix(y_test,prediction)
cm1
#print(train_data)
#for x, y in train_data:
#    print("x", x, "y", y)
#x_train

array([[   0, 1473],
       [   0, 2530]], dtype=int64)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras.layers import Dense


input_data = data_2017
input_data["duration_ms"] = input_data["duration_ms"].astype(float)
input_data["explicit"] = input_data["explicit"].astype(float)
input_data["key"] = input_data["key"].astype(float)
input_data["mode"] = input_data["mode"].astype(float)

# Preprocessing des datas
train_data, test_data = train_test_split(input_data, test_size=0.2)

x_train = train_data.drop(["popularity", "id", "release_date", "name", "artists"], axis=1)
y_train = x_train.pop('top')

x_test = test_data.drop(["popularity", "id", "release_date", "name", "artists"], axis=1)
y_test = x_test.pop('top')

#y_train = np.asarray(train_labels).astype('float32').reshape((-1,1))
#y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))

# List out variable types  (pas utilse ici mais on sait jamais)
data_type_dict = {'numerical': ['instrumentalness', 'key', 'liveness', 'loudness', 'speechiness', 
                                'tempo', 'valence', 'acousticness', 'danceability', 'duration_ms', 'energy'],
                  'categorical': ['mode', 'year', 'explicit'],
                  'text': ['name', 'artists']}
output_var = 'top'

#train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))


print(train_data)
print(type(train_data))


# Define the model
model = keras.Sequential()
tf.keras.backend.set_floatx('float64')
tf.keras.Input(shape=14,)
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='relu'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


#[print(i.shape, i.dtype) for i in model.inputs]
#[print(o.shape, o.dtype) for o in model.outputs]
#[print(l.name, l.input_shape, l.dtype) for l in model.layers]
model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

#_, accuracy = model.evaluate(X, y)
#print('Accuracy: %.2f' % (accuracy*100))